# Introduction to NLP Fundamentals in TensorFlow

NLP is focused on deriving information from natural language (text or speech).

Another common term for NLP problems is sequence to sequence (seq2seq)

In [1]:
## Check for GPU
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-11f57e92-413b-d759-1823-77e1baa3a39d)


In [2]:
# download helper funciton python script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-01-27 17:17:32--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-01-27 17:17:32 (80.0 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
# Import functions from helper_functions.py
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get a text dataset

I am using the the [Kaggle's Introduction to NLP dataset](https://www.kaggle.com/competitions/nlp-getting-started), a set of tweets for binary classifications of disasters.

I will be downloading the dataset from [my course github here](https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip). 

In [4]:
# downloads the dataset from google storage
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-01-27 17:17:40--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.197.128, 74.125.135.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.01s   

2023-01-27 17:17:40 (40.3 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
# Unzip the data
unzip_data("nlp_getting_started.zip")

## Visualizing a text dataset

Need to visualize our text dataset using Python's Pandas.

**NOTE**: A Pandas dataframe can only be the same size as RAM.

In [6]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
# shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
# check out test dataframe 
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# quantity of examples of each class
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

> This is fairly balanced, no balancing required. Otherwise look up imbalanced classification

In [10]:
# Total samples
len(train_df), len(test_df)

(7613, 3263)

In [11]:
# Visualize random training samples
import random
random_index = random.randint(0, len(train_df) - 5) # create random indexes not higher than total samples
for row in train_df_shuffled[["text", "target"]][random_index: random_index + 5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not a real disaster)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target: 0 (not a real disaster)
Text:
SEAN END CAREER sG Blizzard vs KNOCKOUT ... http://t.co/nyv51681uE

---

Target: 0 (not a real disaster)
Text:
I liked a @YouTube video http://t.co/YdgiUYdqgb Mini Pony packs a punch. Live report from the Salem County Fair on CBS3 Eyewitness

---

Target: 0 (not a real disaster)
Text:
@tareksocal I think a lot of celebrities have to treat ppl as if they could harm them

---

Target: 1 (real disaster)
Text:
Service on the Green Line has resumed after an earlier derailment near Garfield with residual delays.

---

Target: 0 (not a real disaster)
Text:
Bright &amp; BLAZING Fireman Birthday Party http://t.co/9rFo9GY3nE #Weddings

---



### Split data into training and validation data set

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
# use train_test_split to split training data into train and validation splits
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                           train_df_shuffled["target"].to_numpy(),
                                                                           test_size=0.1,
                                                                           random_state=42)

In [14]:
# check the lengths of each split
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [15]:
# check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## converting text into numbers

There are multiple ways to convert text to numbers:
* Tokenization - direct mapping of token (word or character) to a number
* Embedding - Creates a vector for each word of arbitrary length (128, 256, 512, etc) and creates a matrix to support showing relationships. The embedding matrix acts as a layer that can be trained as more text is analyzed.

### Text vectorization (tokenization)

In [16]:
# Look at the data
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [17]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in our vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation", # lowercase and removes punctuation
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None, # pads each sequence to be the same length as the longest sequence is
                                    pad_to_max_tokens=False)

In [18]:
# find the average number of tokens (words) in the training tweets
total_words = sum([len(i.split()) for i in train_sentences])

# this produces a float so we round to get an int
average_words = round(total_words / len(train_sentences))

average_words

15

In [19]:
# set up text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be, condenses sentences to this max length

text_vectorizer = TextVectorization(max_tokens=max_vocab_length, # max vocab of 10k words
                                   output_mode="int",
                                   output_sequence_length=max_length) # truncate or pad to match the maximum sequence length of 15 in our case

In [20]:
# Fit the text
text_vectorizer.adapt(train_sentences)

In [21]:
# create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
# choose random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original Text:\n {random_sentence}\n\n Vectorized Version:")
text_vectorizer([random_sentence])

Original Text:
 Amazon Deal - wait or buy? http://t.co/0T8VqKEArI

 Vectorized Version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1329,  711,  637,   53, 1151,    1,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [23]:
# find the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in training data
top_5_words = words_in_vocab[:5] # get most common words
bottom_5_words = words_in_vocab[-5:]

print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer

To make the embedding I will be using the TensorFlow Embedding Layer via `tf.keras.layers.Embedding`

Params for Embedding Layer:
* `input_dim` - Size of the vocabulary
* `output_dim` - the size of the output embedding vector, the vector representation of the word will be this many float values long
* `input_length` - length of sequences being passed to the embedding layer (set to 15 earlier)

In [24]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128,
                             input_length=max_length) # how long is each input

In [25]:
# get random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original Text:\n {random_sentence}\n\nEmbedded Version:\n")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original Text:
 Thought it was a drought @_ASHJ? http://t.co/V4Br5gjMIY

Embedded Version:



<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 2.5300417e-02, -3.9192475e-02, -4.9914766e-02, ...,
         -3.2820329e-03, -4.5645025e-02, -2.0805229e-02],
        [-2.3712862e-02,  1.3817597e-02,  3.1843308e-02, ...,
         -2.0293951e-02,  3.3630971e-02,  2.4327587e-02],
        [ 3.0481521e-02, -1.5789628e-02,  1.3799038e-02, ...,
          3.5035942e-02, -4.2891968e-02, -5.2489340e-05],
        ...,
        [ 4.4674125e-02, -1.0418735e-02,  1.1442997e-02, ...,
         -1.2409963e-02,  2.1032784e-02, -2.3992373e-02],
        [ 4.4674125e-02, -1.0418735e-02,  1.1442997e-02, ...,
         -1.2409963e-02,  2.1032784e-02, -2.3992373e-02],
        [ 4.4674125e-02, -1.0418735e-02,  1.1442997e-02, ...,
         -1.2409963e-02,  2.1032784e-02, -2.3992373e-02]]], dtype=float32)>

> 3 dimensions: 1 sequence, 15 tokens, 128 float values representing each token (word).

In [26]:
# check out single token's embedding
sample_embed[0][0], sample_embed[0][0].shape,  random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.02530042, -0.03919248, -0.04991477, -0.0089213 ,  0.04938241,
         0.03212552, -0.01810931, -0.00484425, -0.02819156, -0.04794249,
         0.04864398,  0.04013126, -0.04621777,  0.0240005 , -0.02970241,
         0.03287921, -0.0494904 ,  0.04103797,  0.02669685, -0.02472763,
         0.02368298,  0.04176605, -0.02795473, -0.04107437,  0.04934213,
        -0.00425757, -0.02122968, -0.04448226,  0.03352305,  0.04436103,
         0.0175509 ,  0.02851078,  0.04015655,  0.03729348, -0.0422639 ,
        -0.04793688, -0.0223188 , -0.01676657,  0.04375928, -0.00388291,
         0.02829948, -0.01606231,  0.0344619 ,  0.01342   , -0.03395762,
        -0.04328334,  0.01078639, -0.03919299, -0.02231017,  0.04931773,
         0.00453148, -0.02528858,  0.01481814,  0.03890653, -0.00025135,
        -0.04703635,  0.04472348, -0.01455658, -0.0395655 ,  0.03891492,
         0.04246986,  0.01096515, -0.04459429,  0.0178494 ,  0.02297791,
  

## modeling a text dataset (series of experiments)

The modeling experiements will be:

* Model 0: Naive Bays (baseline, sci-kit learn)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM Model (RNN)
* Model 3: GRU Model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of the training data (to explore a data constrained scenario)

Steps for modeling:
1. Create a model
2. Build a model
3. Fit a model
4. Evaluate the model

### Model 0: Baseline model

To build the baseline, I'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

> It seems commonplace to use non-DL machine learning models to establish a baseline only moving on to Deep Learning when ready to improve.

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# create tokenization and modeling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
    ("clf", MultinomialNB()) # model the text
])

# fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [28]:
# evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Baseline achieves accuracy of: {baseline_score * 100:.2f}%")

Baseline achieves accuracy of: 79.27%


In [29]:
# Make prediction
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### creating an evaluation function for model experiments

Using the following metrics:
* Accuracy
* Precision
* Recall
* F1-Score

In [30]:
# Fucntion to evaluate: accuracy, precision, recall, F1-Score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_preds):
    """
    Calculates model accuracy, precision, recall, and f1 score for a binary classification model.
    """
    # calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_preds) * 100
    # calculate model precision, recall, and f1-score 'weighted' average (supports label imbalance)
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_preds, average="weighted")
    model_results = {"accuracy": model_accuracy,
                     "precision": model_precision,
                     "recall": model_recall,
                     "f1": model_f1}
    
    return model_results

In [31]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_preds=baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## model 1: a simple dense model

In [32]:
# create a tensorboard callback (create new one for each model)
from helper_functions import create_tensorboard_callback

# create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [33]:
# build a model with the funcitonal api
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string) # one dimensional and of type string only
x = text_vectorizer(inputs) # turn input text into numbers
x = embedding(x) # create an embedding of the numeric inputs
x = layers.GlobalAveragePooling1D(name="global_avg_pool_layer")(x) # condence feature vector for each token to one vector
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, binary outputs = 1 and activation of sigmoid
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [34]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_avg_pool_layer (Glob  (None, 128)              0         
 alAveragePooling1D)                                             
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [35]:
# compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [36]:
# fit the model
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                    experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230127-171751
Epoch 1/5
215/215 [==============================] - 6s 5ms/step - loss: 0.6108 - accuracy: 0.6981 - val_loss: 0.5371 - val_accuracy: 0.7480
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.4416 - accuracy: 0.8178 - val_loss: 0.4702 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3478 - accuracy: 0.8603 - val_loss: 0.4612 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2850 - accuracy: 0.8897 - val_loss: 0.4637 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2385 - accuracy: 0.9124 - val_loss: 0.4785 - val_accuracy: 0.7900


In [37]:
# check results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4785 - accuracy: 0.7900


[0.47853755950927734, 0.7900262475013733]

In [38]:
# make predictions
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [39]:
# check out the probability matrix
model_1_pred_probs[:5]

array([[0.4081801 ],
       [0.79052293],
       [0.99770075],
       [0.11320852],
       [0.11179869]], dtype=float32)

In [40]:
# convert model prediction probabilities to model format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [41]:
# calculate model_1 results
model_1_results = calculate_results(val_labels,
                                    model_1_preds)

model_1_results

{'accuracy': 79.00262467191601,
 'precision': 0.7946291128165941,
 'recall': 0.7900262467191601,
 'f1': 0.7872155322757017}

In [42]:
import numpy as np

# check all values of model_1 against baseline
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False,  True])

> We see that only the F1 score resulted in a higher value over the baseline

## Visualizing learned embeddings

In [43]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [44]:
# model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_avg_pool_layer (Glob  (None, 128)              0         
 alAveragePooling1D)                                             
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [45]:
# get the weight matrix of embedding layer (numerical representation of each token in our training data)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape) # same size as vocab and embedding_dim (output dim of our embedding layer)

(10000, 128)


Tensorflow has a tool called projector: http://projector.tensorflow.org/

Tensorflow embeddings documentation: https://www.tensorflow.org/text/guide/word_embeddings

In [46]:
# create embedding files, this code is in embedding documentation
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

> Manually downloaded the `metadata.tsv` and `vectors.tsv`

## Recurrent Neural Network

RNN's are primarily used for sequence data.

RNN's use the representation of a previous input for a later representation.

## Model 2: LSTM

LSTM = Long Short-Term Memory

Architecture:
```
Input (text) -> Tokenize -> Embedding -> Layers (RNN/Dense) -> Output (label probability)
```

In [47]:
# This kernel is representing the shape incorrectly for LSTM and GRU, this modifies the expected shape of the data
tf.keras.backend.set_image_data_format("channels_last")

# create an LSTM model
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.LSTM(units=64, return_sequences=True)(x) # when stacking RNN cells together, need to return_sequences
x = layers.LSTM(64)(x)
# x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [48]:
# get a summary of model_2
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [49]:
# compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [50]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230127-171804
Epoch 1/5
215/215 [==============================] - 6s 9ms/step - loss: 0.2250 - accuracy: 0.9175 - val_loss: 0.5097 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1591 - accuracy: 0.9404 - val_loss: 0.5726 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1274 - accuracy: 0.9508 - val_loss: 0.7478 - val_accuracy: 0.7795
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1062 - accuracy: 0.9602 - val_loss: 0.8423 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0871 - accuracy: 0.9679 - val_loss: 0.9823 - val_accuracy: 0.7717


In [51]:
# make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[1.01574585e-02],
       [7.95881629e-01],
       [9.99791801e-01],
       [2.17962991e-02],
       [5.28928649e-04],
       [9.99382019e-01],
       [9.47596908e-01],
       [9.99875784e-01],
       [9.99813259e-01],
       [7.15352893e-01]], dtype=float32)

In [52]:
# convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [53]:
# calculate model 2 results
model_2_results = calculate_results(y_true=val_labels, y_preds=model_2_preds)
model_2_results

{'accuracy': 77.16535433070865,
 'precision': 0.7726063176824562,
 'recall': 0.7716535433070866,
 'f1': 0.7699532001851459}

In [54]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [55]:
# Create a GRU model
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
outputs = layers.Dense(1, activation="tanh")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [56]:
# get model 3 summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [57]:
# compile the model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [58]:
# fit model 3
model_3.fit(train_sentences,
            train_labels,
            validation_data=(val_sentences, val_labels),
            epochs=5,
            callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                   "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230127-171817
Epoch 1/5
215/215 [==============================] - 4s 8ms/step - loss: 0.4061 - accuracy: 0.8869 - val_loss: 1.6613 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1971 - accuracy: 0.9439 - val_loss: 0.9280 - val_accuracy: 0.7126
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1597 - accuracy: 0.9603 - val_loss: 1.5556 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1246 - accuracy: 0.9721 - val_loss: 1.7122 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1210 - accuracy: 0.9759 - val_loss: 1.6663 - val_accuracy: 0.7730


In [59]:
# make predictions with GRU model
model_3_pred_probs = model_3.predict(val_sentences)

24/24 [==============================] - 0s 3ms/step


In [60]:
# convert model 3 pred probs to labels
model_3_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [61]:
# calculate model 3 results
model_3_results = calculate_results(y_true=val_labels, y_preds=model_3_preds)
model_3_results

{'accuracy': 77.16535433070865,
 'precision': 0.7726063176824562,
 'recall': 0.7716535433070866,
 'f1': 0.7699532001851459}

In [62]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 4: Bidirectional RNN

In [63]:
# Create a bidirectional RNN
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

In [64]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [65]:
# compile model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [66]:
# fit data
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20230127-171830
Epoch 1/5
215/215 [==============================] - 5s 11ms/step - loss: 0.1123 - accuracy: 0.9712 - val_loss: 0.7959 - val_accuracy: 0.7651
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0672 - accuracy: 0.9753 - val_loss: 0.9273 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0587 - accuracy: 0.9753 - val_loss: 0.9848 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0483 - accuracy: 0.9810 - val_loss: 1.2457 - val_accuracy: 0.7612
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0442 - accuracy: 0.9799 - val_loss: 1.1948 - val_accuracy: 0.7441


In [67]:
# Make predictions
model_4_pred_probs = model_4.predict(val_sentences)

24/24 [==============================] - 1s 3ms/step


In [68]:
# convert pred probs to pre labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))

In [69]:
# Calculate the results of model_4
model_4_results = calculate_results(y_true=val_labels,
                                    y_preds=model_4_preds)
model_4_results

{'accuracy': 74.40944881889764,
 'precision': 0.7455720514146248,
 'recall': 0.7440944881889764,
 'f1': 0.7444631382052128}

## CNN for text and other types of sequences

CNN's are typically for images, images are 2D. Text is 1D, `Conv1D` is what I'll be using

```
Inputs (text) -> Tokenization -> Embedding -> Layer(s) (typically Conv1D + pooling) -> Outputs
```

### Model 5: Conv1D

In [70]:
# Test out embedding layer, Conv1D layer and max pooling layer
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # turn target sequence into embedding
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5,
                        activation="relu",
                        padding="valid")

conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # get the most important feature / get feature with the highest value

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [71]:
# build conv1d model
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64, kernel_size=5, activation="relu", padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

# compile
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# summary of model
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 1)              

In [72]:
# fit
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20230127-171847
Epoch 1/5
215/215 [==============================] - 3s 6ms/step - loss: 0.1274 - accuracy: 0.9590 - val_loss: 0.8613 - val_accuracy: 0.7651
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0761 - accuracy: 0.9739 - val_loss: 1.0343 - val_accuracy: 0.7703
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0619 - accuracy: 0.9766 - val_loss: 1.0314 - val_accuracy: 0.7638
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0584 - accuracy: 0.9772 - val_loss: 1.1571 - val_accuracy: 0.7625
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0518 - accuracy: 0.9780 - val_loss: 1.2099 - val_accuracy: 0.7585


In [73]:
# make some predictions
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[1.6183892e-01],
       [9.3920058e-01],
       [9.9985409e-01],
       [5.6436278e-02],
       [1.4730230e-07],
       [9.9867284e-01],
       [9.2865133e-01],
       [9.9986887e-01],
       [9.9999952e-01],
       [6.9635040e-01]], dtype=float32)

In [74]:
# convert model 5 pred probs to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [75]:
# evaluate model 5
model_5_results = calculate_results(y_true=val_labels,
                                    y_preds=model_5_preds)
model_5_results

{'accuracy': 75.8530183727034,
 'precision': 0.7585993128505528,
 'recall': 0.7585301837270341,
 'f1': 0.7572040956354258}

In [76]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 6: TensorFlow Hub Pretrained Sentence Encoder

In [77]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed([sample_sentence,
                      "When you call universal sentence encoder on a sentence, it converts it to a number"])

print(embed_samples[0][:50])

tf.Tensor(
[-0.01157028  0.0248591   0.02878048 -0.012715    0.03971538  0.0882776
  0.02680984  0.05589836 -0.0106873  -0.00597291  0.00639323 -0.01819518
  0.00030813  0.09105888  0.05874644 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865347 -0.04209306  0.02678981  0.03011006  0.00321069
 -0.00337973 -0.04787357  0.0226672  -0.00985925 -0.04063613 -0.01292092
 -0.04666384  0.05630299 -0.03949255  0.00517686  0.02495829 -0.0701444
  0.02871508  0.04947684 -0.00633979 -0.08960192  0.02807118 -0.00808364
 -0.01360602  0.0599865  -0.10361787 -0.05195374  0.00232954 -0.02332531
 -0.03758105  0.03327728], shape=(50,), dtype=float32)


In [83]:
embed_samples[0].shape

TensorShape([512])

In [84]:
# Create a Keras layer using teh USE pretrained layer from TF Hub
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], # due to variable input length for pretrained model, this is blank for variable acceptance
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [97]:
# Create model using sequential api
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
], name="model_6_USE")

In [98]:
# compile
model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [99]:
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_6 (Dense)             (None, 64)                32832     
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [100]:
# train a classifier on top of pretrained embedding
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                   "tf_hub_sentence_encoder")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20230127-181521
Epoch 1/5
215/215 [==============================] - 4s 14ms/step - loss: 0.5054 - accuracy: 0.7815 - val_loss: 0.4471 - val_accuracy: 0.8005
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.4139 - accuracy: 0.8159 - val_loss: 0.4347 - val_accuracy: 0.8150
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3995 - accuracy: 0.8260 - val_loss: 0.4273 - val_accuracy: 0.8123
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3909 - accuracy: 0.8243 - val_loss: 0.4256 - val_accuracy: 0.8136
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3829 - accuracy: 0.8297 - val_loss: 0.4264 - val_accuracy: 0.8150


In [101]:
# made predictions with USE TF Hub model
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 1s 14ms/step


array([[0.14827354],
       [0.7456085 ],
       [0.9910029 ],
       [0.19793382],
       [0.7106697 ],
       [0.7466158 ],
       [0.9827565 ],
       [0.98062027],
       [0.9385668 ],
       [0.07584741]], dtype=float32)

In [102]:
# convert prediciton probabilities to labels
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [103]:
# calculate model_6 performance results
model_6_results = calculate_results(y_true=val_labels,
                                    y_preds=model_6_preds)

model_6_results

{'accuracy': 81.49606299212599,
 'precision': 0.8182618558292442,
 'recall': 0.8149606299212598,
 'f1': 0.8131230163581015}

In [104]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 7: TF Hub USE except 10% of training data

Transfer learning helps when dataset is small. Replicate `model_6` except with smaller dataset.

Creating the same model as `model_6` using the `tf.keras.models.clone_model()` method. The `clone_model()` method resets all weights for the layers of the model.

> **NOTE:** This created a data leakage.

(#) Create subsets of 10% of the training data
```
train_10_percent = train_df_shuffled[["text", "target"]].sample(frac=0.1, random_state=42)
train_sentences_10_percent = train_10_percent["text"].to_list()
train_labels_10_percent = train_10_percent["target"].to_list()
````

In [135]:
# created better dataset split (no data leakage)
train_10_percent_split = int(0.1 * len(train_sentences))
print(train_10_percent_split)
train_sentences_10_percent = train_sentences[:train_10_percent_split]
train_labels_10_percent = train_labels[:train_10_percent_split]
print(len(train_sentences_10_percent))
print(len(train_labels_10_percent))
train_labels_10_percent.dtype

685
685
685


dtype('int64')

In [136]:
# check the distribution
pd.Series(np.array(train_labels_10_percent)).value_counts()

0    406
1    279
dtype: int64

> The distributions are similar so we can proceed

In [137]:
# build model 
model_7 = tf.keras.models.clone_model(model_6)

# compile the model
model_7.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# get summary
model_7.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_6 (Dense)             (None, 64)                32832     
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [138]:
# fit the model to the 10% training data
model_7_history = model_7.fit(train_sentences_10_percent,
                              train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "tf_hub_sentence_encoder_10_percent")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder_10_percent/20230127-185130
Epoch 1/5
22/22 [==============================] - 3s 44ms/step - loss: 0.6723 - accuracy: 0.6745 - val_loss: 0.6511 - val_accuracy: 0.7192
Epoch 2/5
22/22 [==============================] - 1s 27ms/step - loss: 0.6072 - accuracy: 0.7942 - val_loss: 0.5977 - val_accuracy: 0.7690
Epoch 3/5
22/22 [==============================] - 1s 23ms/step - loss: 0.5374 - accuracy: 0.8058 - val_loss: 0.5439 - val_accuracy: 0.7822
Epoch 4/5
22/22 [==============================] - 0s 23ms/step - loss: 0.4749 - accuracy: 0.8219 - val_loss: 0.5094 - val_accuracy: 0.7690
Epoch 5/5
22/22 [==============================] - 0s 23ms/step - loss: 0.4316 - accuracy: 0.8307 - val_loss: 0.4931 - val_accuracy: 0.7756


In [139]:
# make predictions for model trained on 10 percent of the data
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]

24/24 [==============================] - 1s 10ms/step


array([[0.224515  ],
       [0.60046035],
       [0.8981206 ],
       [0.3633685 ],
       [0.52927065],
       [0.6944396 ],
       [0.8668613 ],
       [0.79560757],
       [0.8116021 ],
       [0.16821048]], dtype=float32)

In [140]:
# turn pred probs into labels
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [141]:
# evaluate model 7
model_7_results = calculate_results(y_true=val_labels,
                                    y_preds=model_7_preds)

model_7_results

{'accuracy': 77.55905511811024,
 'precision': 0.7774694686899193,
 'recall': 0.7755905511811023,
 'f1': 0.7734917819402004}

In [142]:
model_6_results

{'accuracy': 81.49606299212599,
 'precision': 0.8182618558292442,
 'recall': 0.8149606299212598,
 'f1': 0.8131230163581015}

## The speed/score tradeoff

In [146]:
# Make a function to test the time of inference
import time
def pred_timer(model, samples):
  """
  Times how long a model takes to make predictions on samples
  """
  start_time = time.perf_counter() # get start time
  model.predict(samples)
  end_time = time.perf_counter() # get finish time
  total_time = end_time - start_time # total time

  time_per_pred = total_time / len(samples)

  return total_time, time_per_pred

In [147]:
# calculate TF Hub Sentence Encover time per pred
model_6_total_pred_timer, model_6_time_per_pred = pred_timer(model=model_6,
                                                            samples=val_sentences)

model_6_total_pred_timer, model_6_time_per_pred

24/24 [==============================] - 0s 10ms/step


(0.346941859999788, 0.00045530427821494486)

In [148]:
# calculate baseline time per pred
baseline_total_pred_time, baseline_time_per_pred = pred_timer(model_0, val_sentences)

baseline_total_pred_time, baseline_time_per_pred

(0.06488025399994513, 8.51446902886419e-05)